In [2]:
from calendar import EPOCH
from importlib.resources import path
import pathlib
from venv import create
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob
from glob import glob

In [4]:
data_dir = pathlib.Path('imgs')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

#glob으로 리스트 만들고 시각화
iu = list(data_dir.glob('iu/*'))
img = PIL.Image.open(str(iu[100]))
# img.show()

batch_size = 32
img_height = 180
img_width = 180

#훈련세트 비율 나누기
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
#검증세트 "
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names

0


IndexError: list index out of range

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

2022-05-03 16:37:23.021672: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [4]:
new_model = tf.keras.models.load_model('save_model/my_model.h5')
loss, acc = new_model.evaluate(val_ds, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

2022-05-03 16:37:24.311709: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6/6 - 1s - loss: 0.6828 - accuracy: 0.6793 - 561ms/epoch - 94ms/step
복원된 모델의 정확도: 67.93%


In [14]:
predict_img_path = '/Users/ganghaeseong/Desktop/신봉선.jpeg'
img = keras.preprocessing.image.load_img(
    predict_img_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = new_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to iu with a 67.69 percent confidence.
